In [48]:
import cmath
import math

import numpy as np
import scipy.linalg as la
import scipy.sparse as sp
from pfapack import pfaffian as pf

In [25]:
A0 = np.array([
    [0.3+0.2j, 0.6+0.8j, 0.1+0.7j, 0.9+0.4j],
    [0.4+0.2j, 0.2+0.3j, 0.5+0.1j, 0.7+0.6j],
    [0.9+0.2j, 0.1+0.3j, 0.4+0.6j, 0.8+0.2j],
    [0.6+0.6j, 0.7+0.1j, 0.3+0.8j, 0.1+0.5j]])
# A1 = A0.transpose()
# A1
A0 = A0 - A0.transpose()
A0

array([[ 0. +0.j ,  0.2+0.6j, -0.8+0.5j,  0.3-0.2j],
       [-0.2-0.6j,  0. +0.j ,  0.4-0.2j,  0. +0.5j],
       [ 0.8-0.5j, -0.4+0.2j,  0. +0.j ,  0.5-0.6j],
       [-0.3+0.2j,  0. -0.5j, -0.5+0.6j,  0. +0.j ]])

In [38]:
n = A0.shape[0]
L = np.eye(n, dtype=A0.dtype)
for k in range(n-2):
    kp = k + 1 + np.abs(A0[k + 1 :, k]).argmax()
    if kp != k + 1:
            print(k)
            print(kp)
            temp = L[k + 1, 1 : k + 1].copy()
            L[k + 1, 1 : k + 1] = L[kp, 1 : k + 1]
            L[kp, 1 : k + 1] = temp
L

0
2
1
3


array([[1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j]])

In [35]:
k = 0
kp = k + 1 + np.abs(A0[k + 1 :, k]).argmax()
kp

2

In [52]:
A = np.array([
    [0.3+0.2j, 0.6+0.8j, 0.1+0.7j, 0.9+0.4j],
    [0.4+0.2j, 0.2+0.3j, 0.5+0.1j, 0.7+0.6j],
    [0.9+0.2j, 0.1+0.3j, 0.4+0.6j, 0.8+0.2j],
    [0.6+0.6j, 0.7+0.1j, 0.3+0.8j, 0.1+0.5j]])
# A1 = A0.transpose()
# A1
A = A - A.transpose()
A

array([[ 0. +0.j ,  0.2+0.6j, -0.8+0.5j,  0.3-0.2j],
       [-0.2-0.6j,  0. +0.j ,  0.4-0.2j,  0. +0.5j],
       [ 0.8-0.5j, -0.4+0.2j,  0. +0.j ,  0.5-0.6j],
       [-0.3+0.2j,  0. -0.5j, -0.5+0.6j,  0. +0.j ]])

In [53]:
# Check if matrix is square
assert A.shape[0] == A.shape[1] > 0
# Check if it's skew-symmetric
assert abs((A + A.T).max()) < 1e-14

n = A.shape[0]

# Check if we have a complex data type
if np.issubdtype(A.dtype, np.complexfloating):
    householder = pf.householder_complex
elif not np.issubdtype(A.dtype, np.number):
    raise TypeError("pfaffian() can only work on numeric input")
else:
    householder = pf.householder_real

A = np.asarray(A)  # the slice views work only properly for arrays

In [54]:
pfaffian_val = 1.0

for i in range(A.shape[0] - 2):
    # Find a Householder vector to eliminate the i-th column
    v, tau, alpha = householder(A[i + 1 :, i])
    A[i + 1, i] = alpha
    A[i, i + 1] = -alpha
    A[i + 2 :, i] = 0
    A[i, i + 2 :] = 0

    # Update the matrix block A(i+1:N,i+1:N)
    w = tau * np.dot(A[i + 1 :, i + 1 :], v.conj())
    A[i + 1 :, i + 1 :] = A[i + 1 :, i + 1 :] + np.outer(v, w) - np.outer(w, v)

    if tau != 0:
        pfaffian_val *= 1 - tau
    if i % 2 == 0:
        pfaffian_val *= -alpha

pfaffian_val

(-0.37682887362833534-1.1304866208850064j)

In [55]:
A

array([[ 0.        +0.j        , -0.37682887-1.13048662j,
         0.        +0.j        ,  0.        +0.j        ],
       [ 0.37682887+1.13048662j,  0.        +0.j        ,
         0.32506487-0.61521359j,  0.        +0.j        ],
       [ 0.        +0.j        , -0.32506487+0.61521359j,
         0.        +0.j        , -0.55993586+0.51216882j],
       [ 0.        +0.j        ,  0.        +0.j        ,
         0.55993586-0.51216882j,  0.        +0.j        ]])